In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
import collections
from itertools import groupby
import pickle
import re
import matplotlib.colors as mcolors
import warnings
import requests
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from functools import reduce

### Global Variables

In [9]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

colors = [color[4:] for color in mcolors.TABLEAU_COLORS.keys()]
colors.extend([color for color in mcolors.CSS4_COLORS.keys()])
colors.extend(
    [
        'whitish', 'bluish', 'reddish', 'greenish', 'backish', 'greyish',
        'backish', 'purplish', 'yellowish', 'orangish', 'brownish', 'pinkish'
    ]
)



traits_dict = {
    'Life Form':
    [
        'Tree', 'Shrub', 'Bush', 'Ficus', 'Strangler', 'Liana', 'Parasitic', 'Palm', 'Herbaceous'
    ],
    'Trunk':
    [
        'Trunk', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Root':
    [
        'Root', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Latex':
    [
        'Latex'
    ],
    'Phyllotaxis': # Leaf Position
    [
        'Phyllotaxis', 'Alternate', 'Whorled', 'Whorls', 'Opposite'
    ],
    'Leaf Composition':
    [
        'Palmate', 'Pinnate', 'Entire', 'Bi-pinnate'
    ],
    'Crown':
    [
        'Crown'
    ],
    'Stem':
    [
        'Stem', 'Circular', 'Square'
    ],
    'Bark':
    [
        'Bark'
    ],
    'Bark Colour':
    [
        'Bark'
    ],
    'Leaf Shape':
    [
        'Simple', 'Bifoliate', 'Trifoliate', 'Digitized', 'Paripinnate', 'Unipinnate', 'Imperipinnate', 
        'Alternate', 'Bipinnate', 'Pinnate', 'Elliptic', 'Elongate', 'Ovate', 'Round', 'Obovate', 'Lanceolate',
        'Kidney-shaped', 'Heart-shaped', 'Spathulate'
    ],
    'Petiole':
    [
        'Petiole', 'Sessile', 'Petiolated', 'Canaliculate', 'Glands', 'Glandular', 
     'Winged' 'Wings', 'Hairs', 'Hair', 'Translucent'
     ],
    'Leaf Colour':
    [
        'Leaf Colour', 'Leaf Color'
    ],
    'Leaf Blade':
    [
        'Leaf Blade', 'Linear', 'Lanceolate', 'Elliptical', 'Obovate', 'Obtriangular', 
        'Obtriangular', 'Asymmetrical', 'Orbicular', 'Bilobed', 'Lobed', 'Lobes', 'Lobe'
    ],
    'Leaf Base':
    [
        'Leaf Base', 'Rounded', 'Cordate', 'Glands'
    ],
    'Leaf Margin':
    [
        'Margin', 'Smooth', 'Wavy', 'Crenate', 'Toothed', 'Teeth', 'Crenate', 'Serrate'
    ],
    'Leaf Apex':
    [
        'Apex', 'Acuminate', 'Apiculate', 'Mucronate', 'Rounded', 'Emarginated'
    ],
    'Leaf side':
    [
        'Glabrous', 'Pubescent', 'Salt Crystals', 'Scales', 'Woolly', 'Powdery'
    ],
    'Leaf glands':
    [
        'Glands', 'Gland', 'Translucent'
    ],
    'Rachis':
    [
        'Rachis', 'Winged'
    ],
    'Vein':
    [
        'Vein'
    ],
    'Tendril':
    [
        'Tendril'
    ],
    'Spine':
    [
        'Spine', 'Prickle', 'Spines', 'Prickles'
    ],
    'Thornes':
    [
        'Thorn', 'Thornes'
    ],
    'Blade Colour':
    [
        'Blade'
    ],
    'Fruit':
    [
        'Drupe', 'Berry', 'Capsule', 'Pod', 'Follicle', 'Achene', 'Winged', 'Follicle',
        'Pod', 'Nutlet', 'Fruit'
    ],
    'Fruit Shape':
    [
        'locular', 'Globose', 'Flattened', 'Elongate', 'Obovoid', 'Ovate', 'Twisted',
        'Curved', 'Pyriform', 'Ovoid'
    ],
    'Fruit Colour':
    [
        'Fruit'
    ],
    'Inflorescences':
    [
        'Inflorescences', 'Inflorescence', 'Sessile', 'Panicle', 'Flower head', 'Cyme', 'Glomerule', 
        'Fascicle', 'Umbel', 'Corymb', 'Rootlet', 'Spike', 'Dichasium', 'Fascicle',
        'Globose', 'Raceme', 'Fascicle', 'Umbel'
     ],
    'Sexuality':
    [
        'Sexuality', 'Axillary', 'Terminal'
    ],
    'Flower Colour':
    [
        'Flower colour', 'Flower color', 'Flower', 'Flowers'
    ],
    'Flower Shape':
    [
        'Flower shape', 'Petalled', 'Petal', 'Petals', 'Tubular', 'Apetal', 'Butterfly-shaped', 'Shaped', 'Flower', 'Flowers'
    ],
    'Sepal Shape':
    [
        'Sepal', 'Sepals', 'Connate'
    ],
    'Petal Shape':
    [
        'Petal', 'Petals', 'Tepals', 'Tepal', 'Tubular'
    ],
    'Aril Colour':
    [
        'Aril'
    ],
    'Seed Colour':
    [
        'Seed', 
    ],
    'Conspicuousness':
    [
        'Conspicuousness', 'Cryptic'
    ]
}

with open('../../data/supportive/traits_Pierre_and_Andrei.json', 'w') as f:
    json.dump(traits_dict, f)
with open('../../data/supportive/colour_list.json', 'w') as f:
    json.dump(colors, f)

traits_list = list(traits_dict.keys())
traits_list += [trait.lower() for lst in list(traits_dict.values()) for trait in lst]

### Load GT Data

In [5]:
root = "../../data/external/"

#### Data Andrei

In [6]:
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Get Dummies to match DF Pierre
df_Andrei_dummies = pd.get_dummies(df_Andrei.iloc[:, 2:])
# Set species back
df_Andrei_dummies = df_Andrei_dummies.set_index(df_Andrei['Species'])

# Create tuple list for multi index
Andrei_multi_index = []
for top_index in df_Andrei.columns:
    for sub_index in df_Andrei_dummies.columns:
        if top_index in sub_index:

            sub_index = sub_index.split('_')[-1]
            Andrei_multi_index.append((top_index, sub_index))

# Set Mutli index
df_Andrei_dummies.columns = pd.MultiIndex.from_tuples(Andrei_multi_index)
df_Andrei_dummies

Life form      Leaf position                      \
                               liana tree     alternate alternate, opposite   
Species                                                                       
Avicennia germinans                0    1             0                   0   
Metopium brownei                   0    1             1                   0   
Handroanthus billbergii            0    1             0                   0   
Bourreria succulenta               0    1             1                   0   
Bursera karsteniana                0    1             1                   0   
Bursera simaruba                   0    1             1                   0   
Bursera tomentosa                  0    1             1                   0   
Cynophalla flexuosa                1    0             1                   0   
Cynophalla hastata                 0    1             1                   0   
Quadrella indica                   0    1             1                   0   
Quadrella odoratissima             0    1             1                   0   
Crossopetalum rhacoma              0    1             0                   0   
Maytenus tetragona                 0    1             1                   0   
Maytenus versluysii                0    1             1                   0   
Clusia rosea                       0    1             0                   0   
Conocarpus erectus                 0    1             1                   0   
Laguncularia racemosa              0    1             0                   0   
Erythroxylum havanense             0    1             1                   0   
Croton niveus                      0    1             1                   0   
Hippomane mancinella               0    1             1                   0   
Libidibia coriaria                 0    1             1                   0   
Pithecellobium unguis-cati         0    1             1                   0   
Prosopis juliflora                 0    1             1                   0   
Vachellia tortuosa                 0    1             1                   0   
Vitex compressa                    0    1             0                   0   
Vitex cymosa                       0    1             0                   0   
Trichilia trifolia                 0    1             1                   0   
Eugenia procera                    0    1             0                   0   
Guapira pacurero                   0    1             0                   0   
Coccoloba swartzii                 0    1             1                   0   
Coccoloba uvifera                  0    1             1                   0   
Krugiodendron ferreum              0    1             0                   1   
Rhizophora mangle                  0    1             0                   0   
Guettarda roupalifolia             0    1             0                   0   
Randia aculeata                    0    1             0                   0   
Amyris ignea                       0    1             1                   0   
Zanthoxylum monophyllum            0    1             1                   0   
Casearia tremula                   0    1             1                   0   
Schoepfia schreberi                0    1             1                   0   
Jacquinia arborea                  0    1             1                   0   
Guaiacum officinale                0    1             0                   0   
Guaiacum sanctum                   0    1             0                   0   

                                                           \
                           opposite opposite, whorls of 3   
Species                                                     
Avicennia germinans               1                     0   
Metopium brownei                  0                     0   
Handroanthus billbergii           1                     0   
Bourreria succulenta              0                     0   
Bursera karsteniana               0         

#### Data Pierre

In [7]:
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]
df_Pierre = df_Pierre.set_index(df_Pierre['Species']['species'])
df_Pierre

#### Data Palms

In [8]:
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

### RESHAPE DATA FOR PALMS

# Exclude string types
df_Daniels_int = df_Daniel.select_dtypes(exclude=[object])
df_Daniels_str = df_Daniel.select_dtypes(include=[object])
# Drop numbers
df_Daniels_semi_ints = df_Daniels_int.loc[:, df_Daniels_int.max() <= 3]
# Merge again
df_Daniel_edit = pd.merge(df_Daniels_str, df_Daniels_semi_ints, left_index=True, right_index=True)

# Get colors as lst of lsts 
FruitColorDescription_colors_lst = []

for palm_colors in df_Daniels_str['FruitColorDescription'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        FruitColorDescription_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        FruitColorDescription_colors_lst.append([])

MainFruitColors_colors_lst = []

for palm_colors in df_Daniels_str['MainFruitColors'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        MainFruitColors_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        MainFruitColors_colors_lst.append([])

# Init SKlearn MLB
mlb = MultiLabelBinarizer()

# Create dummies for color columns
df_FruitColorDescription = pd.DataFrame(
    {
        'FruitColorDescription': FruitColorDescription_colors_lst
    }, columns=['FruitColorDescription'])

s = df_FruitColorDescription['FruitColorDescription']
df_FruitColorDescription = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('FruitColorDescription', column) for column in df_FruitColorDescription.columns]
df_FruitColorDescription.columns = pd.MultiIndex.from_tuples(columns)

# Create dummies for color columns
df_MainFruitColors = pd.DataFrame(
    {
        'MainFruitColors': MainFruitColors_colors_lst
    }, columns=['MainFruitColors'])

s = df_MainFruitColors['MainFruitColors']
df_MainFruitColors = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour', column) for column in df_MainFruitColors.columns]
df_MainFruitColors.columns = pd.MultiIndex.from_tuples(columns)

df_Daniels_str_non_color = df_Daniels_str[['UnderstoreyCanopy', 'FruitSizeCategorical', 'FruitShape', 'Conspicuousness']]

# df_Daniels_str_non_color.columns = pd.MultiIndex.from_tuples(
#     [
#         ('Crown', 'UnderstoreyCanopy'),
#         ('Fruit Size', 'FruitSizeCategorical'),
#         ('Fruit Shape', 'FruitShape'),
#         ('Conspicuousness', 'Conspicuousness'),
#     ]
# )

df_Daniels_str_non_color_dummies = pd.get_dummies(df_Daniels_str_non_color)
columns = []
for column in df_Daniels_str_non_color_dummies.columns:
    level0, level1 = column.split('_')
    if level0 == 'UnderstoreyCanopy':
        level0 = 'Crown'
    elif level0 == 'FruitSizeCategorical':
        level0 = 'Fruit Size'
    elif level0 == 'FruitShape':
        level0 = 'Fruit Shape'
    elif level0 == 'Conspicuousness':
        level0 = 'Conspicuousness'
    columns.append((level0, level1))
    
df_Daniels_str_non_color_dummies.columns = pd.MultiIndex.from_tuples(columns)

### JOIN ALL DATA
data_frames = [df_FruitColorDescription, df_MainFruitColors, df_Daniels_str_non_color_dummies]
df_Daniel_merged = pd.concat(data_frames, axis=1)
df_Daniel_merged

FruitColorDescription                             \
                                         black blue bluish brown brownish   
SpecName                                                                    
Acanthophoenix crinita                       1    0      0     0        0   
Acanthophoenix rousselii                     1    0      0     0        0   
Acanthophoenix rubra                         1    0      0     0        0   
Acoelorrhaphe wrightii                       1    0      0     1        0   
Acrocomia aculeata                           0    0      0     0        0   
...                                        ...  ...    ...   ...      ...   
Wettinia quinaria                            0    0      0     1        0   
Wettinia radiata                             0    0      0     0        0   
Wettinia verruculosa                         0    0      0     1        0   
Wodyetia bifurcata                           0    0      0     0        0   
Zombia antillarum                            0    0      0     0        0   

                                                                  ...  \
                         chocolate coral crimson darkgreen green  ...   
SpecName                                                          ...   
Acanthophoenix crinita           0     0       0         0     0  ...   
Acanthophoenix rousselii         0     0       0         0     0  ...   
Acanthophoenix rubra             0     0       0         0     0  ...   
Acoelorrhaphe wrightii           0     0       0         0     0  ...   
Acrocomia aculeata               0     0       0         0     1  ...   
...                            ...   ...     ...       ...   ...  ...   
Wettinia quinaria                0     0       0         0     1  ...   
Wettinia radiata                 0     0       0         0     0  ...   
Wettinia verruculosa             0     0       0         0     1  ...   
Wodyetia bifurcata               0     0       0         0     0  ...   
Zombia antillarum                0     0       0         0     0  ...   

                         Fruit Size Fruit Shape                            \
                              small   ellipsoid elongate fusiform globose   
SpecName                                                                    
Acanthophoenix crinita            1           0        0        0       0   
Acanthophoenix rousselii          1           0        0        0       0   
Acanthophoenix rubra              1           0        0        0       0   
Acoelorrhaphe wrightii            1           0        0        0       0   
Acrocomia aculeata                0           0        0        0       0   
...                             ...         ...      ...      ...     ...   
Wettinia quinaria                 1           0        0        0       0   
Wettinia radiata                  0           0        0        0       1   
Wettinia verruculosa              1           0        0        0       0   
Wodyetia bifurcata                0           0        0        0       1   
Zombia antillarum                 1           0        0        0       0   

                                                 Conspicuousness          
                         ovoid pyramidal rounded     conspicuous cryptic  
SpecName                                                                  
Acanthophoenix crinita       0         0       0               0       1  
Acanthophoenix rousselii     1         0       0               0       1  
Acanthophoenix rubra         1         0       0               0       1  
Acoelorrhaphe wrightii       1         0       0               0       1  
Acrocomia aculeata           1         0       0               0       1  
...                        ...       ...     ...             ...     ...  
Wettinia quinaria            1         0       0               0       1  
Wettinia radiata             0         0       0               1       0  
Wettinia verruculos